In [1]:
from src import api as API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Update Files
Sat Jun 22 00:59:10 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.05 Secs
Sat Jun 22 00:59:10 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Sat Jun 22 00:59:10 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Sat Jun 22 00:59:12 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.32 Secs
Sat Jun 22 00:59:16 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.39 Secs
Sat Jun 22 00:59:18 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240620.feather Done! Cost 0.08 Secs
Sat Jun 22 00:59:18 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240621.feather Done! Cost 0.05 Secs
Sat Jun 22 00:59:18 2024 : Updated ~ data\DataBase\DB_models\risk_cov\2024\risk_cov.20240621.feather Done! Cost 0.03 Secs
Sat Jun 22 00:59:18 2024 : Updated ~ data\DataBase\DB_models\risk_spec\2024\risk_spec.20240621.feath

In [2]:
from src import api as API
API.Predictor.update_factors()

Load  2 DataBlocks...... finished! Cost 0.11 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.15 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


In [2]:
from src import api as API
API.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
